##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Search Grounding

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Search_Grounding.ipynb"><img src="../images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook contains a walkthrough of how to use the search grounding feature in the Gemini API.

In [ ]:
!pip uninstall -y -q google-ai-generativelanguage google-generativeai
!pip install -q git+https://github.com/google/generative-ai-python

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 11.6 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
from IPython.display import Markdown, HTML

## Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
genai.configure(api_key=GOOGLE_API_KEY, client_options={'api_endpoint': 'autopush-generativelanguage.sandbox.googleapis.com'})

## Search Grounding

Take a look at the available models from the Gemini API. To perform search grounding, use `'models/gemini-1.5-pro-002'` when calling the API.

In [ ]:
for model in genai.list_models():
  if '002' in model.name:
    print(model.name)

models/gemini-1.5-pro-002
models/gemini-1.5-flash-002


`GoogleSearchRetrieval` is a tool to retrieve public web data for grounding, powered by Google. Part of `GoogleSearchRetrieval` is dynamic search retrieval, which uses real time retrieved data to build answers to queries.

Two possible ways to use include `GoogleSearchRetrieval` are to pass in a string or dictionary to the field. First, you will pass in `google_search_retrieval` to the `tools` parameter in `generate_content`.

In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002')
response = model.generate_content(contents="What is the land area of Spain?",
                                  tools='google_search_retrieval')

Take a look at the first response produced by the model.

In [ ]:
Markdown(response.candidates[0].content.parts[0].text)

Spain's land area is approximately 500,000 square kilometers.  More precisely, slightly different figures are reported by different sources, ranging from 499,733 sq. km to 506,030 sq. km, likely due to variations in measurement methodologies and inclusion/exclusion of certain territories.  This makes it the largest country in Southern Europe, the second largest in Western Europe (after France), and the fourth largest on the European continent (after Russia, Ukraine, and France).

Beyond the mainland, Spain's territory includes the Balearic Islands in the Mediterranean, the Canary Islands in the Atlantic, and the autonomous cities of Ceuta and Melilla in North Africa. These islands and cities add to the overall land area, with the Balearic Islands covering almost 5,000 sq. km and the Canary Islands covering over 7,400 sq km. The cities of Ceuta and Melilla are much smaller, covering around 32 sq km combined.  The inclusion or exclusion of smaller Spanish territories may account for the different land area figures.

It's worth noting that Spain's land use is predominantly dedicated to agriculture, livestock farming, and forestry, which account for about 80% of the country. The remaining 20% includes urban areas, infrastructure, and other land uses.


Now, take a look at the rendered HTML content gathered by the Gemini API.

In [ ]:
HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content)

You will also receive links that direct you to sources used for your query.

In [ ]:
response.candidates[0].grounding_metadata.grounding_chunks

[web {
  uri: "https://vertexaisearch.cloud.sandbox.google.com/grounding-api-redirect/AaZmLhMNkRxr0IUdX6RKVu3PgfN7_E75HECNoXAHn8kp7Gl9LVK7SPs-JdIZyEwwhB_i48XmBEJ4a9XXVKIuD5tZ1FVV2PosgK9jrjzmn3V2TcmXvI7mE3DjvMWcGxpJ_aODO-OWI0qMLs8IsKOB9A=="
  title: "nationsonline.org"
}
, web {
  uri: "https://vertexaisearch.cloud.sandbox.google.com/grounding-api-redirect/AaZmLhOsiqOWYA6HphfLPpYCImiPVvuCasg1WIcOnbcdpGiRt9GS0pslqMBuw9H4WsDGoPmnnXIfQo9_SLSPnNmzj2kIlL24kbuDeI8n5CM6HzKihLIRIF6PgpECsl7P-lEtdmAUJ6H3_3AD1Qj3AVpQlS9ZkhUQE0xMNL8jJ2X7d8gmwTUH8Z-GHoaZBX7D_00DtV89IhLrAg=="
  title: "lamoncloa.gob.es"
}
]

And supporting segments, showing which parts of the output are quored from which source.

In [ ]:
response.candidates[0].grounding_metadata.grounding_supports

[segment {
  end_index: 150
  text: "Spain\'s land area is approximately 505,000 square kilometers, making it the fourth-largest country in Europe and the second-largest in Western Europe."
}
grounding_chunk_indices: 0
confidence_scores: 0.6591796875
, segment {
  start_index: 152
  end_index: 249
  text: "This figure includes the mainland and island territories such as the Balearic and Canary Islands."
}
grounding_chunk_indices: 1
confidence_scores: 0.8894008994102478
]

### Customize search grounding parameters

For a dictionary implementation, you don't actually have to pass in `mode` or `dynamic_threshold` key-value pairs. If you don't pass in a value, a default value is created. See the next code cell for this. For the `mode` value, you can pass in one of two specifications:

1. `unspecified`: Always trigger retrieval.
2. `dynamic`: Run retrieval only when system decides it is necessary.

The `dynamic_threshold` is a float value that acts as a threshold on how relevant answers returned should be. You can tweak this parameter depending on how much variation you want in your answer.

In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002')
response = model.generate_content(contents="What is the land area of Spain?",
                                  tools={"google_search_retrieval": {"dynamic_retrieval_config": {"mode": "MODE_DYNAMIC",
                                                                                                  "dynamic_threshold": 0.5}}})

In [ ]:
Markdown(response.candidates[0].content.parts[0].text)

Spain's land area is approximately 505,000 square kilometers, making it the fourth-largest country in Europe and the second-largest in Western Europe.  This figure includes the mainland and island territories such as the Balearic and Canary Islands.  The mainland itself occupies around 493,500 square kilometers.  It is worth noting that slight variations exist in the reported figures depending on the source and measurement methods.  As of today, September 25th, 2024, this information is considered up-to-date, but it's important to remember that these figures might be revised in the future.


You can also pass in an empty dictionary. Default values for `mode` and `dynamic_threshold` will be used.

In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002')
response = model.generate_content(contents="What is the land area of Spain?",
                                  tools={"google_search_retrieval": {}})

In [ ]:
Markdown(response.candidates[0].content.parts[0].text)

Spain's land area is approximately 505,000 square kilometers.  This makes it the largest country in Southern Europe, the second largest in Western Europe (after France), and the fourth largest on the European continent (after Russia, Ukraine, and France).

It's worth noting that this figure includes the mainland and the island territories. The mainland itself covers about 493,500 square kilometers, while the Balearic and Canary Islands add another 12,500 square kilometers.  Two small Spanish cities, Ceuta and Melilla, located on the North African coast, contribute a small additional area.  Spain also possesses several small, uninhabited enclaves.

Spain is known for its diverse geography, ranging from mountains and plateaus to coastal plains. The central plateau, the Meseta Central, is a prominent feature, divided by mountain ranges and surrounded by other significant mountainous regions like the Pyrenees and the Sierra Nevada. This varied terrain contributes to a wide range of climates and ecosystems across the country.  As of today, September 25, 2024, please remember that these figures might be subject to revision as more precise measurements become available.


## With chat

If you attach the `google_search_retrieval` tool to the model, it can be used in every turn of the chat:

In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002', tools="google_search_retrieval")
chat = model.start_chat()
result = chat.send_message('what is the area of spain?')

In [ ]:
display(Markdown(result.text))

display(HTML(result.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Spain's total area is approximately 505,000 square kilometers, or about 195,000 square miles. This includes the mainland and island territories. The mainland itself covers roughly 504,782 square kilometers (194,897 square miles).  It's worth noting that different sources may vary slightly in their figures.

Here's a breakdown of Spain's territory:

* **Mainland:**  Located on the Iberian Peninsula, shared with Portugal and Andorra.  This is where the majority of Spain's land area and population is located.
* **Island Territories:** These include the Balearic Islands in the Mediterranean Sea and the Canary Islands in the Atlantic Ocean.
* **Exclaves:** Spain also has two exclaves (Ceuta and Melilla) located on the northern coast of Africa. There are also smaller territories or Places of Sovereignty (Plazas de soberanía) located on and off the coast of Morocco. One such place is the small exclave of Llívia within France.

Due to this diverse geography, Spain enjoys a varied climate and landscape. From the Pyrenees mountains to the Mediterranean coast, the country offers a wide array of natural environments.  It's the largest country in Southern Europe and the fourth largest in continental Europe (after Russia, Ukraine, and France).


In [ ]:
result = chat.send_message('what is the google stock price?')


display(Markdown(result.text))

display(HTML(result.candidates[0].grounding_metadata.search_entry_point.rendered_content))

As of September 26, 2024, the price of Alphabet stock (GOOGL) is around $163.  More specifically, it opened at $163.32.  GOOG (Class C shares) is priced at $163.62. These values fluctuate throughout the trading day, so this is a snapshot in time.

It's important to distinguish between the two classes of Alphabet stock:

* **GOOGL (Class A):** These shares have voting rights.
* **GOOG (Class C):** These shares do not have voting rights.

Generally, their prices move very similarly.  For the most up-to-the-minute price, you should consult a live stock ticker.


But you can also pass the tool through the `send_message` call.

In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002')
chat = model.start_chat()
result = chat.send_message('what is the area of spain?', tools="google_search_retrieval")